## WPL Financial Model

#### Notes
1) Need to figure out how to add solar extension period + end effects period
2) Move functions into separate .py file after finishing
3) Dictionary of dfs of depreciations?
4) start thinking about how to automate QC'ing - ask Barett and Rowen about what they do to QC typically
5) Create functions to apply formatting
    - https://pandas.pydata.org/docs/user_guide/style.html
    - also from openpyxl.styles import Font, PatternFill, Alignment
6) automate making tables in "Financial Portfolios" tab --> the ones that are not hard coded

### Import packages & Data

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
from datetime import datetime
import pathlib

# Set the path to the folder containing your Excel files
folder_path = '/Users/alomsadze/OneDrive - Charles River Associates International/Desktop/WPL/Python Version/'
model_inputs = pd.ExcelFile(folder_path + "Direct Model Inputs.xlsx")
model_inputs

In [18]:
financial_scalars_inputs = model_inputs.parse("Financial Inputs", index_col=0)
financial_scalars_inputs

,Value
Scalar Input,
Start Year,2023
Income Tax Rate,0.272431
Federal Income Tax Rate,0.21
State Income Tax Rate,0.079027
License Fee,0.0319
Property Tax Rate,0
Return on Equity (Existing),0.1
Return on Equity (New),0.1
Cost of Debt (Existing),0.039293


In [3]:
aurora_portfolio_summary = model_inputs.parse("Portfolio Summary")
capacity_payments = model_inputs.parse("Capacity Payments")
datacenter_scenario_financials = model_inputs.parse("Datacenter Scenario Financials", header=None)

### Supporting functions -- move to separate python file later

In [4]:
def read_excel_with_tables(df):
    """
    Read a single Excel sheet with multiple tables separated by an empty row.
    Each table is identified by the word "table" in a cell, and its name is in the next cell.

    Parameters:
    - file_path: str, path to the Excel file
    - sheet_name: str, name of the sheet in the Excel file

    Returns:
    - Dictionary of DataFrames, where keys are table names and values are corresponding DataFrames
    """

    # Initialize variables
    tables = {}
    current_table_name = None
    current_table_start_row = None

    # Iterate through rows
    for index, row in df.iterrows():
        if "table" in str(row.values).lower():
            # Save the current table if it exists
            if current_table_name is not None:
                data_table = df.iloc[current_table_start_row:index - 1, :].reset_index(drop=True)
                # Set the column names to be the values in the first row
                data_table.columns = data_table.iloc[0]
                # Drop the first row, which is now redundant as column headers
                data_table = data_table[1:]
                # Drop columns without names
                data_table = data_table.dropna(axis=1, how='all')
                # Now df contains your data with only columns that have column names
                tables[current_table_name] = data_table

            # Update variables for the new table
            current_table_name = str(row[1]) # this takes the value from the cell next to the cell that says "table"
            ########print("current table name is", current_table_name)
            current_table_start_row = index + 1  # Assuming the next row is empty
            
    # Add the last table
    if current_table_name is not None:
        data_table = df.iloc[current_table_start_row:index + 1, :].reset_index(drop=True)
        # Set the column names to be the values in the first row
        data_table.columns = data_table.iloc[0]
        # Drop the first row, which is now redundant as column headers
        data_table = data_table[1:]
        # Drop columns without names
        data_table = data_table.dropna(axis=1, how='all')
        # Now df contains your data with only columns that have column names
        tables[current_table_name] = data_table
        
    return tables


## Revenue Requirement Calculations

In [5]:
# GLOBAL VARIABLES TO GENERALIZE
Run_ID = 'CIC'
Scenario = 'Continue_Change'
year = 2023 
aurora_portfolio_ID = 2
aurora_condition = 'ATC'
case_name = "Datacenter"

In [6]:
# years we want to calculate revenue requirement for
start_year = 2023
end_year = 2072
years = np.arange(start_year, end_year + 1)

### 1. O&M Summary

#### [1.a] Total Portfolio Cost Check

In [7]:
#capacity_payments
#capacity_payments_filtered = capacity_payments[(capacity_payments.Scenarios == Scenario) & (capacity_payments['Case Name'] == case_name)]
#capacity_payments_filtered[2022]

In [8]:
## TOTAL PORTFOLIO COST CHECK

# Total owned resources costs (no market purchases/contracts)
total_owned_cost_yearly = []
# Market purchases cost
market_purchases_yearly = []
# Revenues from market sales
market_sales_yearly = []
# Cost of contracts
contract_costs_yearly = []
# Contract Sales
contract_sales_yearly = []
# High Load Capacity Payments = Cost of short capacity in high load scenario
capacity_payments_yearly = []

condition_mask = aurora_portfolio_summary['Condition'] == aurora_condition
run_id_mask = aurora_portfolio_summary['Run_ID'] == Run_ID
portfolio_id_mask = aurora_portfolio_summary['Portfolio_ID'] == aurora_portfolio_ID
aurora_portfolio_summary_filtered = aurora_portfolio_summary[condition_mask & run_id_mask & portfolio_id_mask]

capacity_payments_filtered = capacity_payments[(capacity_payments.Scenarios == Scenario) & (capacity_payments['Case Name'] == case_name)]

aurora_years = np.sort(aurora_portfolio_summary.Time_Period.unique())

for year in aurora_years:
    # select row for current year
    curr_year_data = aurora_portfolio_summary_filtered.loc[(aurora_portfolio_summary_filtered['Time_Period'] == year)]
    curr_year_capacity_payments = capacity_payments_filtered[year]
    # extract data
    total_owned_cost_yearly.append(curr_year_data["Resource_Cost_Total"].sum() * 1000)
    market_purchases_yearly.append(curr_year_data['Market_Purchases_Cost_Total'].sum() * 1000)
    market_sales_yearly.append(curr_year_data['Market_Sales_Cost_Total'].sum() * 1000)
    contract_costs_yearly.append(curr_year_data['Contract_Purchases_Cost_Total'].sum() * 1000)
    contract_sales_yearly.append(curr_year_data['Contract_Sales_Cost_Total'].sum() * 1000)
    capacity_payments_yearly.append(curr_year_capacity_payments.sum())

    
# Summarize
# Net market purchases = Purchases + Sales
net_market_purchases_yearly = [sum(costs) for costs in zip(market_purchases_yearly, market_sales_yearly)]
total_portfolio_cost_yearly= [sum(costs) for costs in zip(total_owned_cost_yearly, net_market_purchases_yearly,
                                                  contract_costs_yearly, contract_sales_yearly)]
total_portfolio_cost_yearly

# dictionary of lists 
VOM_portfolio_cost_dict = {'Year': aurora_years,
        'Total Portfolio Cost': total_portfolio_cost_yearly, 
        'Total Owned Cost': total_owned_cost_yearly, 
        'Net Market Purchases': net_market_purchases_yearly, 
        'Market Purchases (Energy)': market_purchases_yearly,
        'Market Sales (Energy)': market_sales_yearly, 
        'Contract Cost': contract_costs_yearly, 
        'Contract Sales': contract_sales_yearly,
        'High Load Capacity Payment': capacity_payments_yearly} 
   
VOM_portfolio_cost_df = pd.DataFrame(VOM_portfolio_cost_dict)
VOM_portfolio_cost_df.set_index('Year', inplace=True)
VOM_portfolio_cost_df = VOM_portfolio_cost_df.T
VOM_portfolio_cost_df.style.format(precision=2)

Year,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046
Total Portfolio Cost,360114373.05,315543248.05,254601854.00,266444571.78,297370895.02,316452912.11,335602806.27,340641207.03,325180210.94,329969429.69,320359316.41,334487855.47,251862718.75,256107863.28,264522406.25,278117394.53,294598144.53,319782792.97,339273960.94,357824754.88,378143000.00,392755512.45,333651893.31,341379343.26,267445956.30
Total Owned Cost,537460187.50,471145218.75,353665406.25,262398250.00,253670625.00,269277125.00,293761968.75,308243187.50,325586125.00,245003406.25,229553406.25,228581312.50,267606250.00,267590437.50,263890875.00,268308562.50,276112906.25,270473062.50,286528218.75,293996718.75,333906562.50,333948343.75,350943906.25,232417156.25,234832531.25
Net Market Purchases,-211586578.12,-206615415.04,-155103590.82,-45490605.47,-1956746.09,880042.97,-9773449.22,-21969156.25,-55561976.56,28918558.59,35088441.41,49422453.12,-66399035.16,-52659835.94,-41258140.62,-32482687.50,-24464847.66,5588796.88,11931453.12,51119648.44,31557671.88,54768781.25,-21341078.12,104923101.56,28574812.50
Market Purchases (Energy),27016125.00,16312913.09,15318784.18,37247597.66,57834707.03,64844214.84,63107496.09,53082539.06,48967757.81,86054789.06,90726929.69,100006851.56,42647191.41,49081226.56,54515609.38,60523109.38,64799777.34,82657304.69,91013796.88,120368687.50,116709039.06,133438296.88,130535000.00,197774578.12,175095375.00
Market Sales (Energy),-238602703.12,-222928328.12,-170422375.00,-82738203.12,-59791453.12,-63964171.88,-72880945.31,-75051695.31,-104529734.38,-57136230.47,-55638488.28,-50584398.44,-109046226.56,-101741062.50,-95773750.00,-93005796.88,-89264625.00,-77068507.81,-79082343.75,-69249039.06,-85151367.19,-78669515.62,-151876078.12,-92851476.56,-146520562.50
Contract Cost,39703292.97,56600320.31,61741785.16,55335277.34,51587679.69,52316910.16,53354585.94,54367175.78,55156062.50,56047464.84,55717468.75,56484089.84,50655503.91,41177261.72,41889671.88,42291519.53,42950085.94,43720933.59,40814289.06,12708387.70,12678765.62,4038387.45,4049065.19,4039085.45,4038612.55
Contract Sales,-5462529.30,-5586875.98,-5701746.58,-5798350.10,-5930663.57,-6021166.02,-1740299.19,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
High Load Capacity Payment,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


#### [1.b] Other Fixed Costs

In [9]:
datacenter_scenario_financials_tables = read_excel_with_tables(datacenter_scenario_financials)
list(datacenter_scenario_financials_tables.keys())

['Ongoing CapEx (Nominal$)',
 'FOM (Nominal$)',
 'Decomissioning',
 'Transmission Upgrade Costs - DR Costs',
 'Existing Capital -  Net Book Value BOY',
 'Existing Capital -  Net Book Value EOY',
 'Existing Capital - Tax Value - Federal - BOY',
 'Existing Capital - Tax Value - Federal - EOY',
 'Existing Capital - Tax Value - State - BOY',
 'Existing Capital - Tax Value - State - EOY',
 'Retired',
 'Distribution Offset',
 'Tax Equity Costs',
 'Cumulative Installed Capacity (MW)',
 'Ongoing CapEx ($2021/kW-yr)',
 'Fixed O&M ($2021/kW-yr)',
 'Distribution Offset ($2021/kW)',
 'RT Subhourly and A/S Offset ($2021/kW-yr)']

In [11]:
transmission_upgrade_costs_df = datacenter_scenario_financials_tables['Transmission Upgrade Costs - DR Costs']
FOM_yearly = transmission_upgrade_costs_df[transmission_upgrade_costs_df['Category'] == 'FOM']
FOM_yearly = FOM_yearly.loc[:, FOM_yearly.columns.isin(aurora_years)].values[0]
Transmission_Upgrade_OpEx_yearly = transmission_upgrade_costs_df[transmission_upgrade_costs_df['Category'] == 'Transmission Upgrade OpEx']
Transmission_Upgrade_OpEx_yearly = Transmission_Upgrade_OpEx_yearly.loc[:, Transmission_Upgrade_OpEx_yearly.columns.isin(aurora_years)].values[0]
DSM_Costs_yearly = transmission_upgrade_costs_df[transmission_upgrade_costs_df['Category'] == 'DSM Costs']
DSM_Costs_yearly = DSM_Costs_yearly.loc[:, DSM_Costs_yearly.columns.isin(aurora_years)].values[0]

#  KEEP ADDING ROWS #####################


# dictionary of lists 
FOM_portfolio_cost_dict = {'Year': aurora_years,
                           'FOM': FOM_yearly,
                           'Transmission Upgrade OpEx': Transmission_Upgrade_OpEx_yearly,
                           'DSM Costs': DSM_Costs_yearly} 

FOM_portfolio_cost_dict = pd.DataFrame(FOM_portfolio_cost_dict)
FOM_portfolio_cost_dict.set_index('Year', inplace=True)
FOM_portfolio_cost_dict = FOM_portfolio_cost_dict.T
FOM_portfolio_cost_dict.style.format(precision=2)

Year,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046
FOM,50201214.93,59707421.54,56192856.66,58415753.77,71870573.81,68329970.66,70350835.96,62381445.81,71355624.65,43302290.19,44179982.88,45311957.85,45778741.48,47505085.45,48420670.20,48822429.54,51160694.79,51096581.52,51041855.43,51720035.36,52394896.23,52576138.34,52330266.00,33675490.81,34349000.63
Transmission Upgrade OpEx,0,122425.41,677965.17,677965.17,775528.18,3763883.03,5797956.57,7874745.66,9995147.31,12160077.40,14370471.02,14370471.02,15522573.49,15522573.49,15522573.49,15522573.49,15522573.49,15522573.49,16501402.48,18100417.51,20821408.10,20821408.10,37308402.46,37308402.46,40265257.32
DSM Costs,698000,923260.50,985775.50,1110848,1235920.50,1360993.00,1486065.50,1611138.00,1736210.50,1772670.92,1809897.01,1847904.85,1886710.85,1926331.78,1966784.74,2008087.22,2050257.06,2093312.45,2137272.01,2182154.73,2227979.98,2274767.56,2322537.67,2371310.97,2421108.50


In [31]:
financial_scalars_inputs.loc['Long-term solar projects ITCs or PTCs?'].values[0]

'PTC'